In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "4224"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.getDocuments({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#loads image
image_filename = 'Grazioso Salvare Logo.png'
#encodes into base64
encoded_image = base64.b64encode(open(image_filename, 'rb').read())



app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1("Robey - CS340"))),
    #sets image to center, with 25% of height and width making it more compact and easier to scroll down
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={
                    'height' : '25%',
                    'width' : '25%',

                })),
    html.Hr(),
    html.Div([
        
        #adds 4 required radio buttons and sets first value as all animals/reset
        dcc.RadioItems(id='filter-type',
        options = [{'label':'Water Rescue','value':'Water'},
                   {'label':'Mountain Rescue','value':'Mountain'},
                   {'label':'Disaster Tracking','value':'Disaster'},
                   {'label':'All Animals','value':'reset'}
        ],
        value = 'reset'
        ),

    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#sets up tables for interaction with clientside. imported from module 6
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(radio_filter):
    #checks what filter is being used for the radio 
    #Water = Water Rescue
    if radio_filter == 'Water':
         df = pd.DataFrame(list(shelter.getDocuments({"animal_type":"Dog","breed":{
             "$in":[
                 "Labrador Retriever Mix",
                 "Chesa Bay Retr Mix",
                 "Newfoundland"]
         }, "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},
                                                      "age_upon_outcome_in_weeks":{"$lte":156}})))
    #Mountain = Mountain Rescue
    elif radio_filter == 'Mountain':
        df = pd.DataFrame(list(shelter.getDocuments({"animal_type":"Dog","breed":{
            "$in":[
                "German Shepherd",
                "Alaskan Malamute",
                "Old English Sheepdog",
                "Siberian Husky",
                "Rottweiler"]
        }, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
                                                     "age_upon_outcome_in_weeks":{"$lte":156}})))
    #Disaster = Disaster Relief
    elif radio_filter == 'Disaster':
        df = pd.DataFrame(list(shelter.getDocuments({"animal_type":"Dog","breed":{
            "$in":[
                "Doberman Pinsch",
                "German Shepherd",
                "Golden Retriever",
                "Bloodhound",
                "Rottweiler"]}, "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},
                                                     "age_upon_outcome_in_weeks":{"$lte":300}})))
    #else is reset/all animals
    else:
        df = pd.DataFrame(list(shelter.getDocuments({})))
            
            
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    documents=df.to_dict('records')
       
    return (documents,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    #creates pie graph
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dff, names=dff.breed,)
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    #If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [ 
        dl.Map(style={'width': '500px', 'height': '350px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            #13 and 14 are the locations of the longitude and lattitude.  4 is the location of the dog.
            dl.Marker(position=[dff.iloc[4,13], dff.iloc[4,14]],  children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[4,9]) 
                    
                ])
            ])
                        
        ])
    ]
        
    




app
